<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D8%A7%D8%AE%D8%A8%D8%A7%D8%B1_%D8%B0%D8%A7%D8%AA_%D8%A7%D9%84%D8%AA%D8%A7%D8%AB%D9%8A%D8%B1_%D8%A7%D9%84%D8%B9%D8%A7%D9%84%D9%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي من موقع Investing.com أو أي مصدر موثوق."""
    try:
        url = "https://www.investing.com/economic-calendar/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.select('tr[data-event-type="high"]')  # افترض أن الأخبار ذات التأثير العالي لها خاصية معينة

        high_impact_news = []
        for item in news_items:
            time_str = item.select_one('.event-date').text.strip()
            date_str = item.select_one('.event-time').text.strip()
            event_time = pd.to_datetime(f"{date_str} {time_str}")
            high_impact_news.append(event_time)

        return high_impact_news
    except Exception as e:
        print(f"An error occurred while fetching high impact news: {e}")
        return []


In [ ]:
def is_news_time(high_impact_news):
    """التحقق مما إذا كان الوقت الحالي قريبًا من موعد صدور الأخبار ذات التأثير العالي."""
    current_time = pd.Timestamp.now()
    for news_time in high_impact_news:
        if (news_time - pd.Timedelta(hours=1) <= current_time <= news_time + pd.Timedelta(hours=1)):
            return True
    return False


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # جلب الأخبار ذات التأثير العالي
        high_impact_news = fetch_high_impact_news()

        # التحقق مما إذا كان الوقت الحالي قريبًا من موعد صدور الأخبار
        if is_news_time(high_impact_news):
            print("لا يمكن فتح صفقات بسبب الأخبار ذات التأثير العالي.")
            return

        # باقي الكود كما هو...


In [ ]:
from datetime import datetime, timedelta

def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار الأمريكي من موقع Investing.com."""
    try:
        url = "https://www.investing.com/economic-calendar/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.select('tr[data-impact="high"]')  # اختيار الأخبار ذات التأثير العالي
        news_times = []

        for item in news_items:
            time_str = item.select_one('td[data-time]').get('data-time')
            news_time = datetime.fromtimestamp(int(time_str))
            news_times.append(news_time)

        return news_times
    except Exception as e:
        print(f"An error occurred while fetching high impact news: {e}")
        return []

def is_news_time(news_times):
    """التحقق مما إذا كانت هناك أخبار ذات تأثير عالي خلال ساعة قبل أو بعد الوقت الحالي."""
    current_time = datetime.now()
    for news_time in news_times:
        if news_time - timedelta(hours=1) <= current_time <= news_time + timedelta(hours=1):
            return True
    return False

# تعديل دالة build_strategy
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # جلب الأخبار ذات التأثير العالي
        news_times = fetch_high_impact_news()

        # تحقق مما إذا كانت هناك أخبار قريبة
        if is_news_time(news_times):
            print("لا يمكن فتح صفقات بسبب الأخبار ذات التأثير العالي.")
            return  # إيقاف تنفيذ الصفقات

        # باقي الكود كما هو...
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب آخر صف من البيانات
        last_row = data.iloc[-1]

        # حساب عدد الصفقات المفتوحة
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))

        # الشروط لفتح أوامر الشراء
        if (last_row['close'] < last_row['MA'] and
            last_row['close'] > last_row['Lower_Band'] and
            last_row['close'] < last_row['Middle_Band'] and
            last_row['MACD'] < last_row['Signal_Line'] and
            last_row['RSI'] < 50 and
            last_row['%D'] > last_row['%K'] and
            last_row['Momentum'] < 0.05):
            if open_positions_count < 5:
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع.")

        # الشروط لفتح أوامر البيع
        elif (last_row['close'] > last_row['MA'] and
              last_row['close'] < last_row['Upper_Band'] and
              last_row['close'] > last_row['Middle_Band'] and
              last_row['MACD'] > last_row['Signal_Line'] and
              last_row['RSI'] > 50 and
              last_row['%D'] < last_row['%K'] and
              last_row['Momentum'] > 0.05):
            if open_positions_count < 5:
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        # فتح أوامر بناءً على التنبؤ فقط إذا كانت الشروط متحققة
        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء بناءً على التنبؤ.")
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع بناءً على التنبؤ.")

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")
